## Cleaning the provided example data

**Follow this notebook to understand how to prepare data for taxUMAP.**

*MAKE SURE TO FILL IN THE CELL BELOW WITH A VALID PATH TO YOUR DOWNLOADED DATA.*

In [1]:
# You will need to fill in this next variable
directory_to_tsv_file = '/Users/granthussey/Lab/Schluter/karl_baby/ALL.unoise.vsearch.tsv'
directory_to_data_folder = './'

In [2]:
import os

import numpy as np
import pandas as pd


def fill_taxonomy_table(tax):
    """This fills unknown values in the taxonomy table with a unique value.
    This method of filling missing taxonomic data is recommended for taxumap."""

    taxlevels = list(tax.columns[1::])
    root_level = tax.columns[0]

    tax[root_level] = tax[root_level].fillna("unknown_%s" % root_level)

    for i, level in enumerate(taxlevels):
        _missing_l = tax[level].isna()
        tax.loc[_missing_l, level] = [
            "unknown_%s_of_" % level + str(x)
            for x in tax.loc[_missing_l][taxlevels[i - 1]]
        ]

    for i, (ix, c) in enumerate(tax.iteritems()):
        tax.loc[:, ix] = tax[ix].astype(str) + "____" + tax.index[i]

    tax = tax.applymap(lambda v: v if "unknown" in v else v.split("____")[0])

    return tax


def calculate_relative_counts(counts):
    """From a counts table, calculate relative OTU abundances and include those as new column"""

    relative_counts = counts.groupby("index_column").apply(

        lambda g: pd.DataFrame(
            {
                "RelativeCount": g["Count"] / g["Count"].sum(),
                "OTU": g["OTU"],
                "index_column": g.name,
            }

        )
    )
    c = counts.set_index(["index_column", "OTU"]).join(
        relative_counts.set_index(["index_column", "OTU"])
    )
    c = c.reset_index()

    return c

### `df1` is OTU counts and contains a column of taxonomical information

We will use this to create the `taxonomy` and the `rel_abundances` tables required for taxUMAP.

1. First, we will remove the taxonomy and other excess columns to create an OTU table, `rel_abundances`.
2. Second, we will use the information of the taxonomy column to ceate `taxonomy`.

In [3]:
df1 = pd.read_table(directory_to_tsv_file)
df1

,#OTU,66627277,66627260,66600234,66627266,66627270,66627271,66627280,66627268,66627276,...,66627290,66704977,66704978,66704904,66704907,66704935,66704940,negPCR3,Taxonomy,Centroid
0,Uniq114339,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Firmicutes;,TGGGGAATCTTCCGCAATGGACGAAAGTCTGACGGAGCAACGCCGC...
1,Uniq53046,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Firmicutes;Negativicutes;Selenomonada...,TGGGGAATCTTCCGCAATGGACGAAAGTCTGACGGAGCAACGCCGC...
2,Uniq5707,0,0,0,0,0,0,0,0,0,...,0,47,0,0,1,0,0,0,Bacteria;Firmicutes;Clostridia;Clostridiales;L...,TGGGGAATATTGCACAATGGGGGAAACCCTGATGCAGCGACGCCGC...
3,Uniq45364,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Proteobacteria;Gammaproteobacteria;En...,TGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCCATGCCGC...
4,Uniq80019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Proteobacteria;Gammaproteobacteria;En...,TGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCCATGCCGC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,Uniq103183,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Firmicutes;Bacilli;,TGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCGACGCCGC...
2358,Uniq371,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,TGAGGAATATTGGTCAATGGGCGTAAGCCTGAACCAGCCAAGTCGC...
2359,Uniq75647,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Bacteria;Proteobacteria;,TGGGGAATATTGCACAATGGGCGCAAGCCTGATGCAGCCATGCCGC...
2360,Uniq12824,0,0,0,0,0,0,0,0,0,...,0,0,0,0,22,0,0,0,Bacteria;Firmicutes;Bacilli;Bacillales;Family_...,TAGGGAATCTTCCGCAATGGGCGAAAGCCTGACGGAGCAACGCCGC...


### Now we do step 1, creating `rel_abundances` by subsetting `df1`.

In [4]:
df_otu = df1.rename(columns={'#OTU': 'OTU'}) \
            .set_index('OTU')     \
            .drop(columns=['Centroid', 'Taxonomy']) \
            .transpose() \

df_otu.index.name = 'index_column'
df_otu

OTU,Uniq114339,Uniq53046,Uniq5707,Uniq45364,Uniq80019,Uniq99022,Uniq52090,Uniq93521,Uniq49372,Uniq68685,...,Uniq103475,Uniq106180,Uniq752,Uniq107269,Uniq64359,Uniq103183,Uniq371,Uniq75647,Uniq12824,Uniq82221
index_column,,,,,,,,,,,,,,,,,,,,,
66627277,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66627260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66600234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66627266,0,0,0,0,0,5,0,92,0,0,...,0,0,0,0,0,0,0,0,0,0
66627270,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66704904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66704907,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,22,0
66704935,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,972,0,0,0,0,0


In [5]:
df_counts = pd.DataFrame(df_otu.unstack()) \
    .reset_index() \
    .rename(columns={'level_1': 'SampleID', 0: 'Count'})

df_counts

,OTU,index_column,Count
0,Uniq114339,66627277,0
1,Uniq114339,66627260,0
2,Uniq114339,66600234,0
3,Uniq114339,66627266,0
4,Uniq114339,66627270,0
...,...,...,...
243281,Uniq82221,66704904,0
243282,Uniq82221,66704907,0
243283,Uniq82221,66704935,0
243284,Uniq82221,66704940,0


In [6]:
rel_abundances = calculate_relative_counts(df_counts)
rel_abundances

,index_column,OTU,Count,RelativeCount
0,66627277,Uniq114339,0,0.0
1,66627260,Uniq114339,0,0.0
2,66600234,Uniq114339,0,0.0
3,66627266,Uniq114339,0,0.0
4,66627270,Uniq114339,0,0.0
...,...,...,...,...
243281,66704904,Uniq82221,0,0.0
243282,66704907,Uniq82221,0,0.0
243283,66704935,Uniq82221,0,0.0
243284,66704940,Uniq82221,0,0.0


---

### Now we do step 2, creating `taxonomy` from the taxonomy column in `df1`

In [7]:
df_taxonomy = df1.rename(columns={'#OTU': 'OTU'}) \
    .set_index('OTU')     \
    .loc[:, ['Taxonomy']]


df_taxonomy = df_taxonomy['Taxonomy'].str.split(";", expand=True) \
    .rename(columns={0: 'Kingdom',
                     1: 'Phylum',
                     2: 'Class',
                     3: 'Order',
                     4: 'Family',
                     5: 'Genus',
                     6: 'Other1',
                     7: 'Other2'}).replace(["", 'None', None], value=np.nan)

df_taxonomy = df_taxonomy.drop(columns=['Other1', 'Other2'])
df_taxonomy = fill_taxonomy_table(df_taxonomy)

df_taxonomy

,Kingdom,Phylum,Class,Order,Family,Genus
OTU,,,,,,
Uniq114339,Bacteria,Firmicutes,unknown_Class_of_Firmicutes____Uniq5707,unknown_Order_of_unknown_Class_of_Firmicutes__...,unknown_Family_of_unknown_Order_of_unknown_Cla...,unknown_Genus_of_unknown_Family_of_unknown_Ord...
Uniq53046,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Veillonellaceae,Veillonella
Uniq5707,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Lachnospiraceae_FCS020_group
Uniq45364,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Escherichia-Shigella
Uniq80019,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Escherichia-Shigella
...,...,...,...,...,...,...
Uniq103183,Bacteria,Firmicutes,Bacilli,unknown_Order_of_Bacilli____Uniq45364,unknown_Family_of_unknown_Order_of_Bacilli____...,unknown_Genus_of_unknown_Family_of_unknown_Ord...
Uniq371,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Porphyromonadaceae,Parabacteroides
Uniq75647,Bacteria,Proteobacteria,unknown_Class_of_Proteobacteria____Uniq5707,unknown_Order_of_unknown_Class_of_Proteobacter...,unknown_Family_of_unknown_Order_of_unknown_Cla...,unknown_Genus_of_unknown_Family_of_unknown_Ord...


## Getting ready for taxUMAP

#### Save `df_taxonomy` for taxUMAP

* Have the OTU/ASV be the LAST column, as it is the lowest taxonomical level
* Use `index = False` to not save the numerical index

In [8]:
df_tax_to_save = df_taxonomy.reset_index()

cols = df_tax_to_save.columns.to_list()
cols.append(cols.pop(0))  # Move OTU to last column

df_tax_to_save = df_tax_to_save[cols]
df_tax_to_save.to_csv(os.path.join(
    directory_to_data_folder, 'taxonomy.csv'), index=False)

df_tax_to_save

,Kingdom,Phylum,Class,Order,Family,Genus,OTU
0,Bacteria,Firmicutes,unknown_Class_of_Firmicutes____Uniq5707,unknown_Order_of_unknown_Class_of_Firmicutes__...,unknown_Family_of_unknown_Order_of_unknown_Cla...,unknown_Genus_of_unknown_Family_of_unknown_Ord...,Uniq114339
1,Bacteria,Firmicutes,Negativicutes,Selenomonadales,Veillonellaceae,Veillonella,Uniq53046
2,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Lachnospiraceae_FCS020_group,Uniq5707
3,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Escherichia-Shigella,Uniq45364
4,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Escherichia-Shigella,Uniq80019
...,...,...,...,...,...,...,...
2357,Bacteria,Firmicutes,Bacilli,unknown_Order_of_Bacilli____Uniq45364,unknown_Family_of_unknown_Order_of_Bacilli____...,unknown_Genus_of_unknown_Family_of_unknown_Ord...,Uniq103183
2358,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Porphyromonadaceae,Parabacteroides,Uniq371
2359,Bacteria,Proteobacteria,unknown_Class_of_Proteobacteria____Uniq5707,unknown_Order_of_unknown_Class_of_Proteobacter...,unknown_Family_of_unknown_Order_of_unknown_Cla...,unknown_Genus_of_unknown_Family_of_unknown_Ord...,Uniq75647
2360,Bacteria,Firmicutes,Bacilli,Bacillales,Family_XI,Gemella,Uniq12824


#### Saving `rel_abundances` for taxUMAP

* Should be a numerical array where each row is a sample and each column is an OTU
* The sample ID column should be titled "index_column" for simplest compatability
* Also removed some control samples that are not to be used with taxUMAP (line 5)

In [9]:
df_rel_abun_to_save = rel_abundances.pivot(
    index='index_column', columns='OTU', values='RelativeCount')

df_rel_abun_to_save = df_rel_abun_to_save.fillna(value=0)
df_rel_abun_to_save.drop(index=['negPCR1', 'negPCR2', 'negPCR3'], inplace=True)

df_rel_abun_to_save = df_rel_abun_to_save.reset_index()

df_rel_abun_to_save.to_csv(os.path.join(
    directory_to_data_folder, 'microbiota_table.csv'), index=False)

df_rel_abun_to_save

OTU,index_column,Uniq1,Uniq10,Uniq100,Uniq10005,Uniq100070,Uniq100161,Uniq1003,Uniq100332,Uniq100353,...,Uniq99523,Uniq99645,Uniq99650,Uniq99669,Uniq99681,Uniq99709,Uniq99780,Uniq99833,Uniq99884,Uniq99979
0,66600164,0.670946,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,66600169,0.612890,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,66600209,0.001633,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,66600224,0.006367,0.000000,0.000462,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000024
4,66600226,0.917854,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,66704977,0.000000,0.809957,0.000027,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
96,66704978,0.000000,0.929111,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
97,66704990,0.000000,0.000000,0.000094,0.000000,0.0,0.0,0.003682,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000157
98,66704994,0.004284,0.000000,0.000000,0.000059,0.0,0.0,0.000000,0.0,0.000436,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
